In [30]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

In [31]:
calendar = pd.read_csv("../kaggle_forecasting_accuracy/calendar.csv")
sales_train_validation = pd.read_csv("../kaggle_forecasting_accuracy/sales_train_validation.csv")
sell_prices = pd.read_csv("../kaggle_forecasting_accuracy/sell_prices.csv")

original_sales_train_validation = sales_train_validation

In [32]:
print(sales_train_validation.size)

58510310


In [33]:
unique_products = sales_train_validation['item_id'].unique()
print(len(unique_products))

3049


In [34]:
has_duplicates = sales_train_validation['item_id'].duplicated().any()
print(has_duplicates)

True


<h1>CALENDAR<h1>

In [35]:
pd.set_option('expand_frame_repr', False)
print(calendar)

            date  wm_yr_wk    weekday  wday  month  year       d  event_name_1 event_type_1  event_name_2 event_type_2  snap_CA  snap_TX  snap_WI
0     2011-01-29     11101   Saturday     1      1  2011     d_1           NaN          NaN           NaN          NaN        0        0        0
1     2011-01-30     11101     Sunday     2      1  2011     d_2           NaN          NaN           NaN          NaN        0        0        0
2     2011-01-31     11101     Monday     3      1  2011     d_3           NaN          NaN           NaN          NaN        0        0        0
3     2011-02-01     11101    Tuesday     4      2  2011     d_4           NaN          NaN           NaN          NaN        1        1        0
4     2011-02-02     11101  Wednesday     5      2  2011     d_5           NaN          NaN           NaN          NaN        1        0        1
...          ...       ...        ...   ...    ...   ...     ...           ...          ...           ...          ...      

<h1>SELL PRICES</h1>

In [36]:
print(sell_prices)

        store_id        item_id  wm_yr_wk  sell_price
0           CA_1  HOBBIES_1_001     11325        9.58
1           CA_1  HOBBIES_1_001     11326        9.58
2           CA_1  HOBBIES_1_001     11327        8.26
3           CA_1  HOBBIES_1_001     11328        8.26
4           CA_1  HOBBIES_1_001     11329        8.26
...          ...            ...       ...         ...
6841116     WI_3    FOODS_3_827     11617        1.00
6841117     WI_3    FOODS_3_827     11618        1.00
6841118     WI_3    FOODS_3_827     11619        1.00
6841119     WI_3    FOODS_3_827     11620        1.00
6841120     WI_3    FOODS_3_827     11621        1.00

[6841121 rows x 4 columns]


<h1>SALES TRAIN VALIDATION</h1>

In [37]:
print(sales_train_validation)

                                  id        item_id    dept_id   cat_id store_id state_id  d_1  d_2  d_3  d_4  ...  d_1904  d_1905  d_1906  d_1907  d_1908  d_1909  d_1910  d_1911  d_1912  d_1913
0      HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1       CA    0    0    0    0  ...       1       3       0       1       1       1       3       0       1       1
1      HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1       CA    0    0    0    0  ...       0       0       0       0       0       1       0       0       0       0
2      HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1       CA    0    0    0    0  ...       2       1       2       1       1       1       0       1       1       1
3      HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1       CA    0    0    0    0  ...       1       0       5       4       1       0       1       3       7       2
4      HOBBIES_1_005_CA_1

In [38]:
sales_train_validation = sales_train_validation.drop(['id'], axis = 1) 
sales_train_validation = sales_train_validation.iloc[:, 5:]

In [39]:
sales_train_validation

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [40]:
def MakeTrainAndValidateExamples(example):
    cut_index = int(len(example)*0.8)

    df_train = example.iloc[:cut_index]
    df_validate = example.iloc[cut_index:]


    return df_train, df_validate

<h1>RANDOM FOREST REGRESSION</h1>

In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold

<h2>Sequencing datasets</h2>

In [42]:
def SequencingDatasets(dataset):  
    dataset_seqs = []
    dataset_targets = []
    sequence_len = 30
    for i in range(len(dataset)-sequence_len-1):
        # print(np.array(dataset.iloc[i:i+sequence_len]))
        dataset_seqs.append(np.array(dataset.iloc[i:i+sequence_len]))
        dataset_targets.append(dataset.iloc[i+sequence_len+1])
    dataset_seqs = np.asarray(dataset_seqs)
    dataset_targets = np.asarray(dataset_targets)

    return dataset_seqs, dataset_targets

<h1>Time series</h1>

In [43]:
# # przygotowanie danych
# train_set, validate_set = MakeTrainAndValidateExamples(sales_train_validation.iloc[0])
# train_targets = train_set.values
# validate_targets = validate_set.values

# # konwertowanie danych na szeregi czasowe
# start_date = '2011-01-29'
# end_date = '2016-06-19'
# date_range = pd.date_range(start=start_date, end=end_date, freq='D')
# train_series = pd.Series(train_targets, index=date_range[:len(train_targets)])
# last_value = train_series[-1]
# extra_values = pd.Series([last_value] * (len(validate_series) - 1), index=validate_series.index[1:])
# validate_series = pd.concat([pd.Series([last_value]), extra_values])

# # dopasowanie modelu ARIMA
# model = ARIMA(train_series, order=(3, 1, 2)) # przykładowe wartości parametrów (p,d,q)
# model_fit = model.fit()

# # prognozowanie na zbiorze walidacyjnym
# test_predicted = model_fit.forecast(steps=len(validate_series))

# # obliczenie błędu średniokwadratowego
# test_mse = mean_squared_error(validate_series, test_predicted)
# test_r2 = r2_score(validate_series[:-1], test_predicted[:-1])

# print(f'Test Mean squared error: {test_mse:.2f}')
# print(f'Test R-squared: {test_r2:.2f}')


In [44]:
def train(sales_train_validation_item_for_state_shop):
    # przygotowanie danych
    train_set, validate_set = MakeTrainAndValidateExamples(sales_train_validation_item_for_state_shop)
    train_seqs, train_targets = SequencingDatasets(train_set)
    validate_seqs, validate_targets = SequencingDatasets(validate_set)

    # definiowanie modelu
    rf_model = RandomForestRegressor(random_state=42)

    # zdefiniowanie zakresu hiperparametrów do przeszukania
    param_grid = {'n_estimators': [64, 74, 84, 94, 104, 114, 120],
                  'max_features': ['sqrt', 'log2'],
                  'max_depth': [None, 5, 10, 20],
                  'min_samples_split': [2, 5, 10]}

    # zdefiniowanie metryki
    scoring = 'neg_mean_squared_error'

    # przeszukanie siatki hiperparametrów
    grid_search = GridSearchCV(rf_model, param_grid, scoring=scoring, cv=5)
    grid_search.fit(train_seqs, train_targets)

    # wybór najlepszego modelu
    best_rf_model = grid_search.best_estimator_

    # dokładność modelu na zbiorze walidacyjnym
    test_predicted = best_rf_model.predict(validate_seqs)
    test_predicted_rounded = [round(x) for x in test_predicted]
    test_rmse = mean_squared_error(validate_targets, test_predicted, squared=False)
    test_rmse_rounded = mean_squared_error(validate_targets, test_predicted_rounded, squared=False)
    print(f'Test Root Mean squared error: {test_rmse:.2f}')
    print(f'Test Root Mean squared error for rounded values: {test_rmse_rounded:.2f}')

    return test_rmse, test_predicted, validate_targets

<h3>Rozdzielenie dataframe'u na oddzielne dla każdego stanu i dla każdego sklepu</h3>

In [45]:
dataframes_by_state_and_store = {}
for state, group in original_sales_train_validation.groupby("state_id"):
    dataframes_by_store = {}
    for store, dataframe in group.groupby("store_id"):
        dataframes_by_store[store] = dataframe
    dataframes_by_state_and_store[state] = dataframes_by_store

In [46]:
ca_1_dataframe = dataframes_by_state_and_store['CA']['CA_1']
print(ca_1_dataframe)

                                 id        item_id    dept_id   cat_id store_id state_id  d_1  d_2  d_3  d_4  ...  d_1904  d_1905  d_1906  d_1907  d_1908  d_1909  d_1910  d_1911  d_1912  d_1913
0     HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1       CA    0    0    0    0  ...       1       3       0       1       1       1       3       0       1       1
1     HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1       CA    0    0    0    0  ...       0       0       0       0       0       1       0       0       0       0
2     HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1       CA    0    0    0    0  ...       2       1       2       1       1       1       0       1       1       1
3     HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1       CA    0    0    0    0  ...       1       0       5       4       1       0       1       3       7       2
4     HOBBIES_1_005_CA_1_valid

In [47]:
unique_values = ca_1_dataframe["item_id"].unique()
print(unique_values)
print(len(unique_values))

['HOBBIES_1_001' 'HOBBIES_1_002' 'HOBBIES_1_003' ... 'FOODS_3_825'
 'FOODS_3_826' 'FOODS_3_827']
3049


In [48]:
total_rows = 0

for state in dataframes_by_state_and_store:
    for store in dataframes_by_state_and_store[state]:
        total_rows += len(dataframes_by_state_and_store[state][store])
        
print(total_rows)

30490


In [49]:
for state in dataframes_by_state_and_store:
    for store in dataframes_by_state_and_store[state]:
        dataframes_by_state_and_store[state][store] = dataframes_by_state_and_store[state][store].drop(['id'], axis = 1) 
        # dataframes_by_state_and_store[state][store] = dataframes_by_state_and_store[state][store].iloc[:, [0] + list(range(5, len(dataframes_by_state_and_store[state][store].columns)))]
        dataframes_by_state_and_store[state][store] = dataframes_by_state_and_store[state][store].iloc[:, 5:]
        dataframes_by_state_and_store[state][store] = dataframes_by_state_and_store[state][store].reset_index(drop=True)

In [50]:
for key in dataframes_by_state_and_store:
    print(f"Klucz: {key}, ilość wartości: {len(dataframes_by_state_and_store[key])}")

Klucz: CA, ilość wartości: 4
Klucz: TX, ilość wartości: 3
Klucz: WI, ilość wartości: 3


In [51]:
print(dataframes_by_state_and_store['CA']["CA_1"])

      d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  ...  d_1904  d_1905  d_1906  d_1907  d_1908  d_1909  d_1910  d_1911  d_1912  d_1913
0       0    0    0    0    0    0    0    0    0     0  ...       1       3       0       1       1       1       3       0       1       1
1       0    0    0    0    0    0    0    0    0     0  ...       0       0       0       0       0       1       0       0       0       0
2       0    0    0    0    0    0    0    0    0     0  ...       2       1       2       1       1       1       0       1       1       1
3       0    0    0    0    0    0    0    0    0     0  ...       1       0       5       4       1       0       1       3       7       2
4       0    0    0    0    0    0    0    0    0     0  ...       2       1       1       0       1       1       2       2       2       4
...   ...  ...  ...  ...  ...  ...  ...  ...  ...   ...  ...     ...     ...     ...     ...     ...     ...     ...     ...     ...     ...
3044    0    

<h1>1 przedmiot w jednym stanie i wszystkich sklepach w tym stanie</h1>

In [52]:
# tworzenie pustej listy do przechowywania wyników RMSE
rmse_list = []

# iteracja po wszystkich sklepach w stanie CA

state='CA'
for store in dataframes_by_state_and_store[state]:
    
    # wywołanie funkcji train() dla każdego sklepu
    rmse, test_predicted, validate_targets = train(dataframes_by_state_and_store['CA'][store].iloc[0])
    
    # dodanie wyniku RMSE do listy
    rmse_list.append(rmse)

    print(f'Predicted values for store {store} in {state}: ', test_predicted)
    print(f'Real values for store {store} in {state}: ', validate_targets)

# obliczenie średniej RMSE dla stanu CA
avg_rmse = sum(rmse_list) / len(rmse_list)
print(f"Średnia RMSE dla stanu CA: {avg_rmse}")

Test Root Mean squared error: 0.94
Test Root Mean squared error for rounded values: 0.99
Predicted values for store CA_1 in CA:  [0.57611997 0.48579671 0.58038724 0.4071949  0.51685656 0.52284859
 0.54451393 0.51290569 0.46204604 0.43629836 0.52997869 0.49296301
 0.52250569 0.38379553 0.48414263 0.39229833 0.396832   0.30506716
 0.3238083  0.25682134 0.25560091 0.25316436 0.28914286 0.2238161
 0.16299055 0.09879767 0.02417149 0.02417149 0.02417149 0.02417149
 0.02417149 0.10150183 0.15163932 0.10554105 0.07192669 0.12026918
 0.17348718 0.16120768 0.12025996 0.10484269 0.08455813 0.1385697
 0.20718972 0.25276815 0.30139151 0.26220673 0.33001317 0.40097275
 0.46719377 0.32718982 0.30742492 0.3442047  0.44300289 0.40658668
 0.49155259 0.42775991 0.45439069 0.56925142 0.5024313  0.55998964
 0.50261288 0.51947178 0.47435485 0.63216502 0.56772728 0.57548421
 0.48715556 0.50661975 0.53827216 0.57471531 0.75238925 0.61813788
 0.7104808  0.56279927 0.56429493 0.51995224 0.63364503 0.46687823
 0

In [53]:
#sprawdzenie, czy dobry format
dataframes_by_state_and_store['CA']['CA_1'].iloc[0]

d_1       0
d_2       0
d_3       0
d_4       0
d_5       0
         ..
d_1909    1
d_1910    3
d_1911    0
d_1912    1
d_1913    1
Name: 0, Length: 1913, dtype: int64

<h3>Pomysł na implementację prognowazania po wszystkich wierszach dataframe'ów</h3> (pętla po wszystkich wierszach)

In [55]:
# for index, row in df.iterrows():
#     train(row)